# Company Peer Discovery using Autoencoder

Autoencoders are a neural network used for dimensionality reduction. The neural network learn the latent features of the dataset to transform input features into a compressed representation. 

It can be used for encoding high dimensionality data to reduce noise and compute the similarity between high dimensionality data points.

In this case, we have a list of compnay with their profiles. The problem statement is to find the peer of these companies. 

Traditionally, we would engage an expert and define ruleset to filter and sort these companies into specific buckets which is rather subjective. It would be prohibitive if the list is long, we have 12,491 companies in this list.

We can use autoencoders to encode companies into their latent vector and programmatically search for their peers objectively. Hence, machine learning enabled the discovery of peer companies in a scalable manner.

In this project, you will use an autoencoder to encode company profiles and create a machine learning system for peer discovery. You will go through the applied data science journey:

Data wrangling >> Machine Learning model training >> Serving machine learning model >> evaluating the effectiveness of the machine learning system.

Data file can be downloaded at:
https://drive.google.com/file/d/1FrqsCW758NbZgfKbEoCMDDLmf7kUPf2j/view?usp=sharing



## Adding GPU

We can add a GPU by going to the menu and selecting:

Edit -> Notebook Settings -> Add accelerator (GPU)

Then run the following cell to confirm that the GPU is detected.

In [0]:
!pip uninstall tensorflow -y
!pip install tensorflow-gpu==2.0.0

Uninstalling tensorflow-1.15.0:
  Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 380.8MB 46kB/s 
     |████████████████████████████████| 450kB 27.7MB/s 
     |████████████████████████████████| 3.8MB 46.9MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.9.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


# Data Cleaning and Encoding

In [0]:
import tensorflow as tf
print("Tensorflow version: {}".format(tf.__version__))
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Tensorflow version: 2.0.0
Found GPU at: /device:GPU:0


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving data.f to data.f
User uploaded file "data.f" with length 2068848 bytes


## Read data into dataframe and do some simple cleaning


In [0]:
import pandas as pd
import numpy as np

In [0]:
data_path = 'data.f'
raw_df = pd.read_feather(data_path)

In [0]:
#df preprocessing
ncols_raw = len(raw_df.columns)

#drop duplicate index
raw_df.drop(raw_df.columns[0], axis=1, inplace=True)
raw_df.drop(labels=['year', 'Name', 'OS'], axis=1, inplace=True)
raw_df.replace(to_replace=[None], value=np.nan, inplace=True)

In [0]:
raw_df.head()

,Country,Country Status,Featured Report?,GOLD Community,Listed/Non-listed,Organization type,Region,Sector,Size
0,Italy,OECD,No,No,Listed,Private company,Europe,Energy Utilities,Large
1,United States of America,OECD,No,No,Listed,Private company,Northern America,Healthcare Products,Large
2,United Kingdom of Great Britain and Northern I...,OECD,No,NaN,Non-listed,Subsidiary,Europe,Aviation,Large
3,Sweden,OECD,No,No,Listed,Private company,Europe,Consumer Durables,Large
4,Sweden,OECD,NaN,NaN,NaN,NaN,Europe,Construction Materials,Large


In [0]:
#find columns that has na
raw_df.columns[raw_df.isna().any()]

Index(['Featured Report?', 'GOLD Community', 'Listed/Non-listed',
       'Organization type', 'Size'],
      dtype='object')

In [0]:
raw_df.describe()

,Country,Country Status,Featured Report?,GOLD Community,Listed/Non-listed,Organization type,Region,Sector,Size
count,12491,12491,9802,9114,11833,11730,12491,12491,12458
unique,130,6,2,2,3,7,6,39,3
top,Mainland China,OECD,No,No,Listed,Private company,Asia,Financial Services,Large
freq,1337,6392,9788,8897,7018,8423,4488,1481,7468


In [0]:
#fill na with majorituy category
raw_df.fillna({'Featured Report?':'No', \
               'GOLD Community': 'No', \
               'Listed/Non-listed': 'Listed', \
               'OS':'No', \
               'Organization type': 'Private Company', 
               'Size': 'Large'}, inplace=True)

In [0]:
n_countries = raw_df[raw_df.columns[0]].unique().size
print("Total number of countries: {}".format(n_countries))

Total number of countries: 130


In [0]:
data_clean = raw_df.loc[:, 'Country Status':'Size'].copy()
countries = raw_df.loc[:, 'Country'].unique()

## Encode data

In [0]:
#Creates feature tokens for each feature (Hot-One-Encoding)
features_dataframe = data_clean.copy()
for feature in data_clean.columns:
      dfDummies = pd.get_dummies(data_clean[feature], prefix = feature)
      features_dataframe = pd.concat([features_dataframe, dfDummies], axis=1)

In [0]:
features_dataframe.head()

,Country Status,Featured Report?,GOLD Community,Listed/Non-listed,Organization type,Region,Sector,Size,Country Status_DAC-LDC,Country Status_DAC-LMICT,Country Status_DAC-OLIC,Country Status_DAC-UMICT,Country Status_Non-OECD / Non-DAC,Country Status_OECD,Featured Report?_No,Featured Report?_Yes,GOLD Community_No,GOLD Community_Yes,Listed/Non-listed_Listed,Listed/Non-listed_Non-listed,Listed/Non-listed_Not applicable,Organization type_Cooperative,Organization type_Non-profit organization,Organization type_Partnership,Organization type_Private Company,Organization type_Private company,Organization type_Public institution,Organization type_State-owned company,Organization type_Subsidiary,Region_Africa,Region_Asia,Region_Europe,Region_Latin America & the Caribbean,Region_Northern America,Region_Oceania,Sector_=,Sector_Agriculture,Sector_Automotive,Sector_Aviation,Sector_Chemicals,Sector_Commercial Services,Sector_Computers,Sector_Conglomerates,Sector_Construction,Sector_Construction Materials,Sector_Consumer Durables,Sector_Energy,Sector_Energy Utilities,Sector_Equipment,Sector_Financial Services,Sector_Food and Beverage Products,Sector_Forest and Paper Products,Sector_Healthcare Products,Sector_Healthcare Services,Sector_Household and Personal Products,Sector_Logistics,Sector_Media,Sector_Metals Products,Sector_Mining,Sector_Non-Profit / Services,Sector_Other,Sector_Public Agency,Sector_Railroad,Sector_Real Estate,Sector_Retailers,Sector_Technology Hardware,Sector_Telecommunications,Sector_Textiles and Apparel,Sector_Tobacco,Sector_Tourism/Leisure,Sector_Toys,Sector_Universities,Sector_Waste Management,Sector_Water Utilities,Size_Large,Size_MNE,Size_SME
0,OECD,No,No,Listed,Private company,Europe,Energy Utilities,Large,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,OECD,No,No,Listed,Private company,Northern America,Healthcare Products,Large,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,OECD,No,No,Non-listed,Subsidiary,Europe,Aviation,Large,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,OECD,No,No,Listed,Private company,Europe,Consumer Durables,Large,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,OECD,No,No,Listed,Private Company,Europe,Construction Materials,Large,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
ncolumns_data = len(data_clean.columns) 
ncolumns_code = len(features_dataframe.columns) - ncolumns_data

print("Original data columns: {}".format(ncolumns_data))
print("Encoding columns: {}".format(ncolumns_code))

Original data columns: 8
Encoding columns: 69


In [0]:
# Removes encoding from the list of variables, the operation 
code_columns = list(features_dataframe.columns)[ncolumns_data:]

In [0]:
# Takes only the columns which have been one-hot-encoded
training_df = features_dataframe[code_columns].copy() 

In [0]:
training_df.head(2)

,Country Status_DAC-LDC,Country Status_DAC-LMICT,Country Status_DAC-OLIC,Country Status_DAC-UMICT,Country Status_Non-OECD / Non-DAC,Country Status_OECD,Featured Report?_No,Featured Report?_Yes,GOLD Community_No,GOLD Community_Yes,Listed/Non-listed_Listed,Listed/Non-listed_Non-listed,Listed/Non-listed_Not applicable,Organization type_Cooperative,Organization type_Non-profit organization,Organization type_Partnership,Organization type_Private Company,Organization type_Private company,Organization type_Public institution,Organization type_State-owned company,Organization type_Subsidiary,Region_Africa,Region_Asia,Region_Europe,Region_Latin America & the Caribbean,Region_Northern America,Region_Oceania,Sector_=,Sector_Agriculture,Sector_Automotive,Sector_Aviation,Sector_Chemicals,Sector_Commercial Services,Sector_Computers,Sector_Conglomerates,Sector_Construction,Sector_Construction Materials,Sector_Consumer Durables,Sector_Energy,Sector_Energy Utilities,Sector_Equipment,Sector_Financial Services,Sector_Food and Beverage Products,Sector_Forest and Paper Products,Sector_Healthcare Products,Sector_Healthcare Services,Sector_Household and Personal Products,Sector_Logistics,Sector_Media,Sector_Metals Products,Sector_Mining,Sector_Non-Profit / Services,Sector_Other,Sector_Public Agency,Sector_Railroad,Sector_Real Estate,Sector_Retailers,Sector_Technology Hardware,Sector_Telecommunications,Sector_Textiles and Apparel,Sector_Tobacco,Sector_Tourism/Leisure,Sector_Toys,Sector_Universities,Sector_Waste Management,Sector_Water Utilities,Size_Large,Size_MNE,Size_SME
0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
#Save File 
training_df.to_csv("encoded_data.csv")

# TF Dataset definition

In [0]:
import numpy as np
import tensorflow as tf
import os

np.random.seed(1)
tf.random.set_seed(1)

In [0]:
batch_size = 128

In [0]:
train_array = training_df.to_numpy(dtype=np.float32,copy=True)

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_array)

In [0]:
train_dataset = train_dataset.batch(batch_size=batch_size)

In [0]:
train_dataset = train_dataset.shuffle(train_array.shape[0])

In [0]:
train_dataset = train_dataset.prefetch(batch_size*4)

# Model Definition

## Encoder

In [0]:
intermediate_dim = 23
original_dim = 69

In [0]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, intermediate_dim, original_dim):
    super(Encoder, self).__init__()
    hidden_dim_1 = int((original_dim + intermediate_dim)/2)
    self.hidden_layer_1 = tf.keras.layers.Dense(
      units=hidden_dim_1,
      activation=tf.nn.relu,
      kernel_initializer='he_uniform'
    )
    self.hidden_layer_2 = tf.keras.layers.Dense(
      units=intermediate_dim,
      activation=tf.nn.relu,
      kernel_initializer='he_uniform'
    )
    self.output_layer = tf.keras.layers.Dense(
      units=intermediate_dim,
      activation=tf.nn.sigmoid
    )
    
  def call(self, input_features):
    activation = self.hidden_layer_1(input_features)
    activation = self.hidden_layer_2(activation)
    return self.output_layer(activation)

## Decoder

In [0]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, intermediate_dim, original_dim):
    super(Decoder, self).__init__()
    hidden_dim_2 = int((original_dim + intermediate_dim)/2)
    self.hidden_layer_1 = tf.keras.layers.Dense(
      units=intermediate_dim,
      activation=tf.nn.relu,
      kernel_initializer='he_uniform'
    )
    self.hidden_layer_2 = tf.keras.layers.Dense(
      units=hidden_dim_2,
      activation=tf.nn.relu,
      kernel_initializer='he_uniform'
    )
    self.output_layer = tf.keras.layers.Dense(
      units=original_dim,
      activation=tf.nn.sigmoid
    )
  
  def call(self, code):
    activation = self.hidden_layer_1(code)
    activation = self.hidden_layer_2(activation)
    return self.output_layer(activation)

## Autoencoder

In [0]:
class Autoencoder(tf.keras.Model):
  def __init__(self, intermediate_dim, original_dim):
    super(Autoencoder, self).__init__()
    self.encoder = Encoder(intermediate_dim=intermediate_dim, original_dim=original_dim)
    self.decoder = Decoder(
      intermediate_dim=intermediate_dim,
      original_dim=original_dim
    )
  
  def call(self, input_features):
    code = self.encoder(input_features)
    reconstructed = self.decoder(code)
    return reconstructed

# Model and Training Setup

In [0]:
autoencoder = Autoencoder(
  intermediate_dim=intermediate_dim,
  original_dim=original_dim
)

In [0]:
learning_rate = 0.5e-2
opt = tf.optimizers.Adam(learning_rate=learning_rate)

In [0]:
def loss(model, original):
  reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(model(original), original)))
  return reconstruction_error
  
def train(loss, model, opt, original):
  with tf.GradientTape() as tape:
    gradients = tape.gradient(loss(model, original), model.trainable_variables)
  gradient_variables = zip(gradients, model.trainable_variables)
  opt.apply_gradients(gradient_variables)

# Training

In [0]:
epochs = 50
save_path = "drive/My Drive/model_checkpoints"

In [0]:
def make_folder(path):
  try:
    os.mkdir(path)
  except FileExistsError:
    print("Directory already exist.")
  return

In [0]:
make_folder(save_path)

In [0]:
save_freq = 5
writer = tf.summary.create_file_writer('{}/tmp'.format(save_path))
with writer.as_default():
  with tf.summary.record_if(True):
    for epoch in range(epochs):
      for step, batch_features in enumerate(train_dataset):
        train(loss, autoencoder, opt, batch_features)
        loss_values = loss(autoencoder, batch_features)
        original = batch_features
        reconstructed = autoencoder(tf.constant(batch_features))
        tf.summary.scalar('loss', loss_values, step=step)
        tf.summary.write('original', original, step=step)
        tf.summary.write('reconstructed', reconstructed, step=step)
      
      if epoch%save_freq == 0:
        tf.print("Epoch: {}".format(epoch))
        tf.print(" . Loss:", loss_values)
        path = "{}/epoch_{:03d}".format(save_path, epoch)
        make_folder(path)
        autoencoder.save_weights(path)


Epoch: 0
 . Loss: 0.0375696197
Epoch: 5
 . Loss: 0.0161237791
Epoch: 10
 . Loss: 0.0139395762
Epoch: 15
 . Loss: 0.0127674649
Epoch: 20
 . Loss: 0.00927975401
Epoch: 25
 . Loss: 0.00968246628
Epoch: 30
 . Loss: 0.013391952
Epoch: 35
 . Loss: 0.0115917251
Epoch: 40
 . Loss: 0.00980878342
Epoch: 45
 . Loss: 0.00984678417


# Capstone Project Statement

## Instructions:
1. Create a copy of this collab notebook and suffix it with your name
2. Run through the code example above
3. Work on the project objectives below inside the copy of the notebook

## 1. Use the autoencoder as a basis to create a machine learning model to find the Top 5 similar company given an target company. 

Note: Implement an API/function to query for Top 5 similar company given an target company. Assuming the target company is within the data.f file.

## 2. Improve on the company encoder by using more features or use a different architecture to improve performance of the Top 5 similar company API.